In [25]:
import math

import tabula
import pandas as pd
import numpy as np

In [2]:
tables = tabula.read_pdf('table.pdf', pages=[4,5,6,7], multiple_tables=True)

Got stderr: Aug 26, 2020 1:55:15 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Aug 26, 2020 1:55:16 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Aug 26, 2020 1:55:17 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Aug 26, 2020 1:55:17 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Aug 26, 2020 1:55:17 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Aug 26, 2020 1:55:17 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



# Data Cleaning

In [3]:
# Manually inspect output tables
for table in tables:
    print(table.info())
    
# Index 0-4 are our main tables. Tabel 5 is smaller table on page 7.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  70 non-null     object
 1   Unnamed: 1  70 non-null     object
 2   Unnamed: 2  70 non-null     object
 3   Unnamed: 3  71 non-null     object
 4   ∆G, kJ/mol  71 non-null     object
 5   Unnamed: 4  71 non-null     object
dtypes: object(6)
memory usage: 3.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  71 non-null     object
 1   Unnamed: 1  71 non-null     object
 2   Unnamed: 2  71 non-null     object
 3   Unnamed: 3  72 non-null     object
 4   ∆G, kJ/mol  72 non-null     object
 5   Unnamed: 4  72 non-null     object
dtypes: object(6)
memory usage: 3.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to

In [4]:
# Work with the first table
first_table_copy = tables[0].copy()

In [5]:
first_table_copy.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,"∆G, kJ/mol",Unnamed: 4
0,NaN,NaN,NaN,CODESSA-PRO,TRAIL (Type 1),TRAIL (Type 2)
1,no.,name of compound,exptl,calcd exptl - calcd,calcd exptl - calcd,calcd exptl - calcd
2,1,carbon tetrachloride,-12.56,-12.43 -0.13,-12.56 0.0,-12.56 0.0
3,2,chloroform,-8.16,-11.09 2.93,-8.16 0.0,-8.16 0.0
4,3,methanol,2.8,4.91 -2.11,1.05 1.75,0.89 1.91


In [6]:
# See that there are some problems with the header

# Construct a header
data_label = tables[0].iloc[0]
data_type = tables[0].iloc[1]

column_names = []
for i in range(len(data_label)):
    try:
        math.isnan(data_label[i])
        column_names.append(F"{data_type[i]}")
    except TypeError:
        column_names.append(F"{data_label[i]} {data_type[i]}")
        

print(column_names)

['no.', 'name of compound', 'exptl', 'CODESSA-PRO calcd exptl - calcd', 'TRAIL (Type 1) calcd exptl - calcd', 'TRAIL (Type 2) calcd exptl - calcd']


In [7]:
# More cleaning of headers

for i in range(len(column_names)):
    column_name = column_names[i]
    found = column_name.find('exptl - calcd')
    if found != -1:
        column_name = column_name[:found]
    
    column_names[i] = column_name

print(column_names)

['no.', 'name of compound', 'exptl', 'CODESSA-PRO calcd ', 'TRAIL (Type 1) calcd ', 'TRAIL (Type 2) calcd ']


In [11]:
# Now we will get rid of those two rows we don't need

# First drop first two rows
first_table_copy = first_table_copy.drop(index=[0,1])

# Then set the headers
first_table_copy.columns = column_names

In [12]:
first_table_copy

,no.,name of compound,exptl,CODESSA-PRO calcd,TRAIL (Type 1) calcd,TRAIL (Type 2) calcd
2,1,carbon tetrachloride,-12.56,-12.43 -0.13,-12.56 0.0,-12.56 0.0
3,2,chloroform,-8.16,-11.09 2.93,-8.16 0.0,-8.16 0.0
4,3,methanol,2.8,4.91 -2.11,1.05 1.75,0.89 1.91
5,4,acetonitrile,1.54,3.79 -2.25,1.54 0.0,1.54 0.0
6,5,acetaldehyde,3.65,0.21 3.44,1.58 2.07,2.81 0.84
...,...,...,...,...,...,...
66,65,benzaldehyde,-10.16,-13.15 2.99,-8.79 -1.37,-9.41 -0.75
67,66,benzoic acid,-12.13,-13.73 1.60,-10.37 -1.76,-11.64 -0.49
68,67,4-hydroxybenzaldehyde,-9.99,-11.81 1.82,-9.29 -0.70,-9.90 -0.09
69,68,4-hydroxybenzoic acid,-12.54,-12.20 -0.34,-10.87 -1.67,-12.13 -0.41


In [16]:
# Next split and get rid of value we don't need (we can calculate it ourselves)
for i in range(3,6):
    first_table_copy.iloc[:,i] = first_table_copy.iloc[:, i].str.split().str[0]

In [17]:
first_table_copy

,no.,name of compound,exptl,CODESSA-PRO calcd,TRAIL (Type 1) calcd,TRAIL (Type 2) calcd
2,1,carbon tetrachloride,-12.56,-12.43,-12.56,-12.56
3,2,chloroform,-8.16,-11.09,-8.16,-8.16
4,3,methanol,2.8,4.91,1.05,0.89
5,4,acetonitrile,1.54,3.79,1.54,1.54
6,5,acetaldehyde,3.65,0.21,1.58,2.81
...,...,...,...,...,...,...
66,65,benzaldehyde,-10.16,-13.15,-8.79,-9.41
67,66,benzoic acid,-12.13,-13.73,-10.37,-11.64
68,67,4-hydroxybenzaldehyde,-9.99,-11.81,-9.29,-9.90
69,68,4-hydroxybenzoic acid,-12.54,-12.20,-10.87,-12.13


In [18]:
# Let's do this in a loop then concatenate the tables.

df = pd.DataFrame()

for table in tables[:-1]:
    table_copy = table.copy()
    
    # Construct a header
    data_label = table_copy.iloc[0]
    data_type = table_copy.iloc[1]

    column_names = []
    for i in range(len(data_label)):
        try:
            math.isnan(data_label[i])
            column_names.append(F"{data_type[i]}")
        except TypeError:
            column_names.append(F"{data_label[i]} {data_type[i]}")
    
    for i in range(len(column_names)):
        column_name = column_names[i]
        found = column_name.find('exptl - calcd')
        if found != -1:
            column_name = column_name[:found]

        column_names[i] = column_name
    
    # First drop first two rows
    table_copy = table_copy.drop(index=[0,1])

    # Then set the headers
    table_copy.columns = column_names
    
    
    # Next split and get rid of value we don't need (we can calculate it ourselves)
    for i in range(3,6):
        table_copy.iloc[:,i] = table_copy.iloc[:, i].str.split().str[0]
    
    df = pd.concat([df, table_copy])
    

In [20]:
# Set index to molecule numberr
df.set_index('no.', inplace=True)

In [29]:
# Set 'b' values to NaN
df.replace('b', np.nan, inplace=True)

In [30]:
# Save as csv
df.to_csv('table_from_paper.csv')

In [24]:
# Leave this table for later.

tables[4]

,no.,X,(∆X,t-test,R 2,R cv 2,Unnamed: 0,Unnamed: 1,s 2,Unnamed: 2,descriptor
0,0,-3.062 × 10+00,2.383 × 10+00,-1.285,NaN,NaN,NaN,NaN,NaN,intercept,NaN
1,1,-8.902 × 10+00,4.096 × 10-01,-21.733,0.392,0.380,NaN,NaN,16.44,average complementary information content (ord...,NaN
2,2,2.905 × 10+00,1.953 × 10-01,14.876,0.542,0.529,NaN,NaN,12.45,"LUMO energy,",LUMO
3,3,1.081 × 10-01,9.349 × 10-03,11.567,0.657,0.643,NaN,NaN,9.37,"WNSA-1-weighted PNSA (PNSA1*TMSA/1000), WNSA",NaN
4,4,4.333 × 10+02,4.410 × 1001,9.823,0.693,0.680,NaN,NaN,8.41,"HACA-2/TMSA (Mopac PC), HACA",NaN
5,5,-5.009 × 10+01,7.567 × 10+00,-6.619,0.747,0.732,NaN,NaN,6.97,maximum partial charge (Zefirov) for all atom ...,NaN
6,6,-8.733 × 10+00,1.634 × 10-02,-5.346,0.774,0.759,NaN,NaN,6.26,"H-acceptors FPSA (version 2), FPSA",NaN
7,7,1.401 × 1001,2.890 × 10+00,4.848,0.796,0.779,NaN,NaN,5.66,"ZX shadow/ZX rectangle,",SZXR
